In [14]:
# IMPORTS

import pandas as pd

from textblob import TextBlob

import re
import string

# NLTK
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [2]:
df = pd.read_csv('../data/dataset_twitter-scraper_2023-02-16_09-48-43-259.csv')

C:\Users\Tristan\AppData\Local\Temp\ipykernel_10044\3499226996.py:1: DtypeWarning: Columns (7,8,9,10,11,24,60,98,99,101,102,104,105,107,108,110,111,113,114,116,117,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/dataset_twitter-scraper_2023-02-16_09-48-43-259.csv')


In [13]:
df.head()

,conversation_id,created_at,favorite_count,full_text,hashtags/0,hashtags/1,hashtags/2,hashtags/3,hashtags/4,hashtags/5,...,user_mentions/47/id_str,user_mentions/47/name,user_mentions/47/screen_name,user_mentions/48/id_str,user_mentions/48/name,user_mentions/48/screen_name,user_mentions/49/id_str,user_mentions/49/name,user_mentions/49/screen_name,view_count
0,1625495400470958082,2023-02-14T14:01:18.000Z,410055,"Happy Valentine’s Day to the one and only, @Mi...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16532336.0
1,1625143593286438912,2023-02-13T14:43:21.000Z,28215,"Congratulations to the Kansas City @Chiefs, Pa...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2775676.0
2,1623489922438156288,2023-02-09T01:12:15.000Z,9131,@KingJames has been changing the game for 20 y...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,492644.0
3,1625903851168579585,2023-02-16T06:21:12.000Z,674,@RokoMijic !!,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,443693.0
4,1622770990710235136,2023-02-07T01:35:28.000Z,13821,The scale of devastation after the earthquakes...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3117489.0


#### Data preprocessing

Here we define a function that will help us pre-process our data, this includes cleaning the text data, tokenizing, and removing stop words, punctuation and special characters.

In [16]:
nltk.download('stopwords')

def clean_text(text):
    # Remove URLs
    text = re.sub(r'https\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase 
    text = text.lower()
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text_tokens = word_tokenize(text)

    filtered_text = [word for word in text_tokens if word not in stop_words]

    return " ".join(filtered_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tristan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

In [17]:
# Cleaning the text data
df['clean_text'] = df['full_text'].apply(clean_text)

In [18]:
# getting the sentiment score for each Tweet
df['sentiment'] = df['clean_text'].apply(get_sentiment)

In [22]:
df.head(30)

,conversation_id,created_at,favorite_count,full_text,hashtags/0,hashtags/1,hashtags/2,hashtags/3,hashtags/4,hashtags/5,...,user_mentions/47/screen_name,user_mentions/48/id_str,user_mentions/48/name,user_mentions/48/screen_name,user_mentions/49/id_str,user_mentions/49/name,user_mentions/49/screen_name,view_count,clean_text,sentiment
0,1625495400470958082,2023-02-14T14:01:18.000Z,410055,"Happy Valentine’s Day to the one and only, @Mi...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16532336.0,happy valentine ’ day one make every day feel ...,0.800000
1,1625143593286438912,2023-02-13T14:43:21.000Z,28215,"Congratulations to the Kansas City @Chiefs, Pa...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2775676.0,congratulations kansas city patrick mahomes tr...,0.526667
2,1623489922438156288,2023-02-09T01:12:15.000Z,9131,@KingJames has been changing the game for 20 y...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,492644.0,changing game 20 years become leader court con...,-0.400000
3,1625903851168579585,2023-02-16T06:21:12.000Z,674,@RokoMijic !!,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,443693.0,,0.000000
4,1622770990710235136,2023-02-07T01:35:28.000Z,13821,The scale of devastation after the earthquakes...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3117489.0,scale devastation earthquakes türkiye syria al...,0.000000
5,1625725531336638464,2023-02-16T06:14:04.000Z,1319,@BillyM2k @cb_doge Well I would hope that most...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83061.0,well would hope san franciscans agree principle,0.000000
6,1620905250742730756,2023-02-01T22:01:41.000Z,12351,At my last Black History Month celebration at ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1133433.0,last black history month celebration white hou...,-0.013333
7,1620905250742730756,2023-02-01T22:01:41.000Z,45433,Black History Month is about the shared experi...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3302865.0,black history month shared experience black am...,-0.111111
8,1626097497109311495,2023-02-16T06:07:30.000Z,633,"@wongmjane Yeah, it would be *crazy* to make a...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82777.0,yeah would crazy make ai like irl …,-0.600000
9,1626097497109311495,2023-02-16T05:57:38.000Z,11043,Sounds eerily like the AI in System Shock that...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1845927.0,sounds eerily like ai system shock goes haywir...,0.000000
